In [2]:
import requests
import pandas as pd
from collections.abc import MutableMapping

In [6]:
# set up the initial parameters for the API request
url = "https://lawis.at/lawis_api/public/incident"
params = {
    "startDate": "1900-01-01",
    "endDate": "2023-03-07",
}
headers = {"Accept": "application/json"}

# make the initial request and get the incident IDs
response = requests.get(url, params=params, headers=headers)
incident_ids = [incident["id"] for incident in response.json()]

# initialize an empty list to store the incident data
incident_data = []

# define a recursive function to flatten the incident data
def flatten_dict(d, parent_key='', sep='_'):
    """
    Recursively flattens a nested dictionary into a flat dictionary by combining nested keys
    with a separator to create unique keys.

    Parameters:
        d (dict): The dictionary to flatten.
        parent_key (str): The prefix to add to flattened keys.
        sep (str): The separator to use between keys in the flattened dictionary.

    Returns:
        dict: The flattened dictionary.
    """
    items = []
    for k, v in d.items():
        # combine the current key with the parent key using the separator
        new_key = parent_key + sep + k if parent_key else k

        # if the value is a dictionary, recursively call the function to flatten it
        if isinstance(v, MutableMapping):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            # if the value is not a dictionary, add it to the list of items as a tuple
            items.append((new_key, v))
    # convert the list of items back to a dictionary and return it
    return dict(items)

for incident_id in incident_ids:
    url = f"https://lawis.at/lawis_api/public/incident/{incident_id}"
    response = requests.get(url, headers=headers)
    incident = response.json()
    
    # flatten all dictionaries into separate columns
    flat_incident = flatten_dict(incident)
    
    # add the flattened incident to the list
    incident_data.append(flat_incident)

# convert the incident data to a pandas DataFrame
df = pd.DataFrame(incident_data)

df.to_csv('data/avalanche_data.csv', index=False)

In [4]:
df = pd.read_csv('data/avalanche_data.csv')


     id  valid_time                       date              reported_date  \
0  9349        True  2018-02-17T10:49:00+01:00  2018-02-17T15:02:44+01:00   
1  9350        True  2018-02-17T14:30:00+01:00  2018-02-17T18:20:50+01:00   
2  9379        True  2018-02-24T14:00:00+01:00  2018-02-26T08:01:55+01:00   
3  9353        True  2018-02-15T13:04:00+01:00  2018-02-18T11:20:21+01:00   
4  9354        True  2018-02-17T11:34:00+01:00  2018-02-19T08:00:14+01:00   

           reported_name reported_email  involved_dead  involved_injured  \
0                    lwd            NaN            0.0               0.0   
1  Beobachter Obertauern            NaN            0.0               1.0   
2          SLF LWD Davos            NaN            1.0               3.0   
3                SLP HZS            NaN            1.0               0.0   
4              LWD Tirol            NaN            0.0               0.0   

   involved_uninjured  involved_sweeped  ...  involved_equipment_lvs_text  \
0  